In [262]:
class get_tweets:
    tweepy = __import__('tweepy')
    pd = __import__('pandas')
    def __init__(self):
        print('get_tweets V.0.1 \nImported tweepy,pandas packages')
    def connect(self,consumer_key,consumer_secret,access_token,access_token_secret):
        consumer_key = consumer_key
        consumer_secret = consumer_secret
        access_token = access_token
        access_token_secret = access_token_secret
        auth = self.tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self.api = self.tweepy.API(auth)
    def tweet_to_df(self,tweet):
        fields = ['created_at','id', 'full_text','display_text_range', "entities",'in_reply_to_status_id',
                  'in_reply_to_user_id','coordinates','user','retweet_count','favorite_count']
        fields_dict={}
        for field in fields:
            fields_dict[field] = getattr(tweet, field)
        temp_tweet = self.pd.DataFrame.from_dict(fields_dict, orient='index').T
        return temp_tweet
    def get_original_tweets(self,user_id):
        tweets_df = self.pd.DataFrame()
        for tweet in self.tweepy.Cursor(self.api.user_timeline, id=user_id, tweet_mode='extended').items(200):
            print(tweet.id)
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                single_tweet=self.tweet_to_df(tweet)
                tweets_df = self.pd.concat([tweets_df, single_tweet], ignore_index=True)
        earliest_tweet = min(tweets_df['id'])
        while True:
            for tweet in self.tweepy.Cursor(self.api.user_timeline, id=user_id, max_id=earliest_tweet, tweet_mode='extended').items(200):
                print(tweet.id)
                if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                    single_tweet=self.tweet_to_df(tweet)
                    tweets_df = self.pd.concat([tweets_df, single_tweet], ignore_index=True)
            new_earliest = min(tweets_df['id'])
            if new_earliest == earliest_tweet:
                break
            else:
                earliest_tweet = new_earliest
            print(f'Number of original Tweets got so far:{len(tweets_df)}')
        tweets_df['hashtags'] = [d.get('hasthags') for d in tweets_df.entities]
        tweets_df['user_mentions'] = [d.get('user_mentions') for d in tweets_df.entities]
        tweets_df['followers_count'] = [d.followers_count for d in tweets_df.user]
        tweets_df.drop(['user','entities'], axis=1, inplace=True)
        return tweets_df

In [263]:
ins = get_tweets()

get_tweets V.0.1 
Imported tweepy,pandas packages


In [264]:
ins.connect('9fNwXla6gTMXBlL7P5LCbyKhU',
            '03rznfCOH4twqCkc7tZHeIXikZ4D1sY6rH3j3Fw3nESYDxHmrb',
            '977303768558718978-rXF69Ll2f6pLwHBDMg25K3yO4H8G79T',
            'UbbdQWpbBzLjOb0Z4Ef7trJG7SVXwDmYxYTRvbjSm780q')

In [266]:
trump=ins.get_original_tweets(25073877)

1205492350094630914
1205489214546235393
1205485784968630272
1205483006405808128
1205470941393936384
1205468357610409984
1205459180229935109
1205458677681008640
1205457765961011201
1205455803378999297
1205455275953729536
1205368801438707713
1205341246937600000
1205338851797024770
1205338632573333505
1205338255832555520
1205334980055961600
1205320725692919808
1205320724799508480
1205320724031954944


In [252]:
trump.sort_values('created_at',inplace=True)